In [1]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler

chat = ChatOpenAI(
    max_tokens=1000,
    temperature=0.1,
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()]
)

chef_prompt = ChatPromptTemplate.from_messages([
    ("system", "당신은 세계적인 수준의 국제적인 셰프입니다. 당신은 어떤 종류의 음식이든 찾기 쉬운 재료를 이용해서 쉽게 따라할 수 있는 음식 레시피를 만듭니다."),
    ("human", "저는 {cuisine} 음식을 만들고 싶습니다. 어떻게 만드나요?")
])

chef_chain = chef_prompt | chat

In [2]:
veg_chef_prompt = ChatPromptTemplate.from_messages([
    ("system", "당신은 채식주의자를 위한 셰프입니다. 전통적인 채식주의자용 레시피에 특화되어 있습니다. 당신은 채식주의자용 레시피가 아닌 레시피를 채식주의자용 대안 레시피로 변경해야 합니다 그리고 준비해야 할 것을 알려줍니다. 당신은 제공된 레시피를 너무 많이 바꿔서는 안됩니다. 만약 제공한 음식의 레시피를 채식주의자용 레시피 대안이 없다면 당신은 할 수 없다고 말하면 됩니다."),
    ("human", "{recipe}"),
])

veg_chain = veg_chef_prompt | chat

final_chain = {"recipe": chef_chain} | veg_chain

final_chain.invoke({
    "cuisine": "Indian",
})

인도 음식은 매우 다양하고 맛있는데, 그 중에서도 버터 치킨은 매우 인기 있는 요리입니다. 버터 치킨은 부드럽고 풍부한 버터 소스에 담백한 치킨이 들어있어 맛있는 요리입니다. 아래는 쉽게 만들 수 있는 버터 치킨 레시피입니다.

**인도식 버터 치킨 레시피**

**재료:**
- 500g 닭가슴살
- 1컵 요거트
- 2큰술 레몬 주스
- 2큰술 생강 마늘 페이스트
- 1큰술 코리앤더 가루
- 1큰술 케이프 라임 주스
- 소금, 후추
- 2큰술 식용유
- 1큰술 마가린

**소스:**
- 1컵 토마토 소스
- 1컵 생크림
- 1큰술 생강 마늘 페이스트
- 1큰술 생강 파우더
- 1큰술 코리앤더 가루
- 1큰술 케이프 라임 주스
- 소금, 후추
- 2큰술 버터

**만들기:**
1. 닭가슴살을 작은 조각으로 자른 후, 요거트, 레몬 주스, 생강 마늘 페이스트, 코리앤더 가루, 케이프 라임 주스, 소금, 후추를 넣고 잘 섞어 1시간 이상 재웁니다.
2. 닭을 꼬치에 꽂고 180도로 예열된 오븐에서 20분간 구웁니다.
3. 소스를 만들기 위해 토마토 소스, 생크림, 생강 마늘 페이스트, 생강 파우더, 코리앤더 가루, 케이프 라임 주스, 소금, 후추를 섞어줍니다.
4. 팬에 버터를 녹인 후 소스를 넣고 끓여줍니다.
5. 구워진 닭을 소스에 넣고 마가린을 넣어 마무리해줍니다.
6. 밥이나 나안, 로티와 함께 내놓아 즐기세요.

이렇게 간단하게 만들 수 있는 인도식 버터 치킨은 가족과 친구들과 함께 즐기기에 좋은 요리입니다. 맛있게 만들어보세요!채식주의자용 대안 레시피로 변경할 수 있는 것은 "닭가슴살" 부분입니다. 대신에 대체재료로 사용할 수 있는 것은 대부분 채식주의 식단에 적합한 재료로 대체할 수 있습니다. 따라서 다음과 같이 변경할 수 있습니다:

**채식주의자용 버터 치킨 대안 레시피**

**재료:**
- 500g 토푸(두부) 또는 대파(마른 두부)
- 1컵 대두 요거트
- 2큰술 레몬 주스
- 2큰술 생강 마늘 페이스트
- 1큰술 코리앤더 가루
- 1큰술 케이

AIMessageChunk(content='채식주의자용 대안 레시피로 변경할 수 있는 것은 "닭가슴살" 부분입니다. 대신에 대체재료로 사용할 수 있는 것은 대부분 채식주의 식단에 적합한 재료로 대체할 수 있습니다. 따라서 다음과 같이 변경할 수 있습니다:\n\n**채식주의자용 버터 치킨 대안 레시피**\n\n**재료:**\n- 500g 토푸(두부) 또는 대파(마른 두부)\n- 1컵 대두 요거트\n- 2큰술 레몬 주스\n- 2큰술 생강 마늘 페이스트\n- 1큰술 코리앤더 가루\n- 1큰술 케이프 라임 주스\n- 소금, 후추\n- 2큰술 식용유\n- 1큰술 마가린\n\n**소스:**\n- 1컵 토마토 소스\n- 1컵 코코넛 밀크\n- 1큰술 생강 마늘 페이스트\n- 1큰술 생강 파우더\n- 1큰술 코리앤더 가루\n- 1큰술 케이프 라임 주스\n- 소금, 후추\n- 2큰술 버터\n\n**만들기:**\n1. 토푸(두부) 또는 대파(마른 두부)를 작은 조각으로 자른 후, 대두 요거트, 레몬 주스, 생강 마늘 페이스트, 코리앤더 가루, 케이프 라임 주스, 소금, 후추를 넣고 잘 섞어 1시간 이상 재웁니다.\n2. 토푸(두부) 또는 대파(마른 두부)를 오븐이나 프라이팬에서 구웁니다.\n3. 소스를 만들기 위해 토마토 소스, 코코넛 밀크, 생강 마늘 페이스트, 생강 파우더, 코리앤더 가루, 케이프 라임 주스, 소금, 후추를 섞어줍니다.\n4. 팬에 버터를 녹인 후 소스를 넣고 끓여줍니다.\n5. 구워진 토푸(두부) 또는 대파(마른 두부)를 소스에 넣고 마가린을 넣어 마무리해줍니다.\n6. 밥이나 나안, 로티와 함께 내놓아 즐기세요.\n\n이렇게 채식주의자용으로 변경된 버터 치킨 레시피를 즐겨보세요!')